In [1]:


%load_ext autoreload
%autoreload 2
%load_ext line_profiler


import numpy as np
import DemandEstimation_v2 as DE
from tqdm.notebook import tqdm
from statsmodels.regression.linear_model import OLS 
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt



#### Setting true parameters and simulating one period market

In [2]:
# setting parameters 

f=2 # Firms
j=2# Products
k=2 # Traits (of each product) 
data = DE.endog_data()
X, prod=data.simulate_product_traits(f,j,k, which_type = "Identical")
X = X.reshape(j*f, -1)

# True Parameters 
alpha = -0.02                # Price sensitivity
beta= np.array([1,2])        # Preference for traits 
mc = np.array([60,60, 70 ,40])           # Marginal costs of producing each product
theta = np.append(alpha,beta)  

# Simulating data: 
P = np.zeros((f*j)) +1               #Initital price vector
data.sim_data(P, theta, X, mc, prod, f, j)


TypeError: sim_data() missing 1 required positional argument: 'J'

In [ ]:
# I could add shocks to production costs over time (ar(1) + noise). That would induce variation over time in prices. 
# I think when MC is perfectly known we lose one dimension of noise. Hence what is left is noise wrt. to demand. But the distribution is known and hence create the logit demand system. 

#### Simulating multiple markets

In [5]:
# setting parameters 

f=2 # Firms
j=2# Products
k=2 # Traits (of each product) 

data = DE.endog_data()
X, prod=data.simulate_product_traits(f,j,k, which_type = "Differentiated")
X = X.reshape(j*f, -1)

# True Parameters 
alpha = -2                # Price sensitivity
beta= np.array([2,2])        # Preference for traits 
theta = np.append(alpha,beta)  
cov = np.array([[1,0.5],[0.5,1]])
#cov = np.array([[1,0.0],[0.0,1]])
gamma = np.array([1,1])
T = 10
P0 = np.zeros((f*j)) +1               #Initital price vector

#mc=data.mc_sim(X, gamma,T=T, noise_type = "Triangular")

In [6]:
#P, S = data.sim_data_T(X, f, j, T, theta, gamma, cov, prod)
#S.min()
S = np.array(0) 
while S.min() == 0.0:
    P, S = data.sim_data_T(X, f, j, T, theta, gamma, cov, prod)
    print(S.min()==0.0)


TypeError: sim_data_T() missing 1 required positional argument: 'H'

#### Using OLS to solve the system: 
1) Divide by outside option and Log transform the system. 

In [ ]:
P[:,9]

In [ ]:
# OLS: 
# But I do not have the outside option in my data... I can construct it easily. 
S0=(1-np.sum(S,axis=0))

index=np.log(S/S0).reshape(-1,1)

# OLS with endogenous prices
P1 = P.reshape(-1,1)
XX = np.hstack((P1, X.repeat(T, axis=0)))
#XX = XX[:,0:2]
resultsp=OLS(index,XX).fit()
resultsp.params

# OLS with MC instead



#### Using 2-step IV: 

In [ ]:
check = np.array(np.arange(j*f))
Z = []
for firm, js in prod.items():
    not_f_prods=[j for j in check if j not in js ]
    
    Z_i = X[not_f_prods,:].mean(axis=0)
    
    Z.append(Z_i)
    
Z = np.array(Z) 

ZZ = Z.repeat(T, axis=0)
Piv1 = P.reshape(-1)


results1step=OLS(P1,ZZ).fit()
results1step.params

EP = (ZZ @ results1step.params).reshape(-1,1)

# Step 2 IV: 
ZX = np.hstack((EP, X.repeat(T, axis=0)))
#XX = XX[:,0:2]
resultsIV=OLS(index,ZX).fit()
resultsIV.params



### Doing a simulation study: 

In [350]:
def sim_study(Q,cov, Type = "OLS", Consts = False):
    """
    Type - options "OLS", "IV"
    """

    # No. of Firms, products and traits: 
    f=2 # Firms
    j=3 # Products
    k=3 # Traits (of each product) 
    
    # True Parameters 
    alpha = -5                      # Price sensitivity
    beta= np.array([3,2,1])         # Preference for traits 
    theta = np.append(alpha,beta)   
    gamma = np.array([1,2,3])       # Production costs of traits 

    #cov = np.array([[1,0.5],[0.5,1]])
    T = 100

    if Type == "OLS":
        if Consts == True: 
            parameters = np.zeros((Q,1 + k*2 + f*j*2)) + np.nan
        else:
            parameters = np.zeros((Q,k + 1)) + np.nan

    elif Type == "IV":
        parameters = np.zeros((Q, k+1))

    for q in tqdm(range(Q)): 
        try:
            
            data = DE.endog_data()
            X, prod=data.simulate_product_traits(f,j,k, which_type = "Differentiated")
            H = data.hadamard(f*j,prod)
            X = X.reshape(j*f, -1)

            # Simulate data 
            S = np.array(0) 
            while S.min() == 0.0:
                P, S = data.sim_data_T(X, f, j, T, theta, gamma, cov, prod, H, disable = True)
            S.min()

            S0=(1-np.sum(S,axis=0))

            index=np.log(S/S0).reshape(-1,1)

            #Estimation: 
            if Type == "OLS": 

                # OLS with endogenous prices
                P1 = P.reshape(-1,1)

                if Consts == True: 
                    j_const = np.identity(f*j).repeat(T, axis=0)
                    XX = np.hstack((P1, X.repeat(T, axis=0),j_const))
                else: 
                    XX = np.hstack((P1, X.repeat(T, axis=0)))

                resultsp=OLS(index,XX).fit()
                results = resultsp.params
            
                parameters[q,:] = results
            
            elif Type == "IV":
                parameters[q,:] = data.blp_regression(index, P, X, prod, [f,j,T])

                if np.abs(parameters[q,0]) > 20: 
                    print(parameters[q,0])
                    print("P", P.max(), P.min())
                    print("S", S.max(), S.min())
                
        except np.linalg.LinAlgError: 
            print("linalg error")
            break
    return parameters




    

In [343]:
#DE.first_step_IV(y,[j,T])

# No. of Firms, products and traits: 
f=2 # Firms
j=3 # Products
k=3 # Traits (of each product) 
    
# True Parameters 
alpha = -5                      # Price sensitivity
beta= np.array([2,1,1])         # Preference for traits 
theta = np.append(alpha,beta)   
gamma = np.array([1,1,1])       # Production costs of traits 

cov = np.array([[1,0.5],[0.5,1]])
T = 150


data = DE.endog_data()

X, prod=data.simulate_product_traits(f,j,k, which_type = "Differentiated")
H = data.hadamard(f*j,prod)
X = X.reshape(j*f, -1)

P, S = data.sim_data_T(X, f, j, T, theta, gamma, cov, prod, H, disable = True)

S0=(1-np.sum(S,axis=0))

y=np.log(S/S0).reshape(-1,1)


params, _ = data.first_step_IV(y,[f,j,T])






### BLP-approach in a simple logit model

In [344]:
y_const, _=data.first_step_IV(y,[f,j,T])

ZS = data.blp_instruments(X,prod, [f,j,T])

step2=OLS(P.reshape(-1,1), ZS).fit()
preds=ZS @step2.params 

step3vars = np.hstack((preds.reshape(-1,1), X.repeat(T, axis=0)))

OLS(y_const, step3vars).fit().params


array([-4.97136395,  1.69766141,  0.74424737,  1.23831832])

In [ ]:
S[:,S.argmax(axis=1)].max(axis=0)


#### BLP approach. Do the following regression:
$\delta_{jm} =\beta V(X, p_{jm} ) + \psi_{jm}$ 

This regression is intractable so instead estimate by IV. 

##### Comment: I do not need to simulate new data. I can just run the two approaches on the same data

In [355]:

Q=2000
cov = np.array([[1,0.50],[0.50,1]])
corr=sim_study(Q, cov, Type = "OLS", Consts = False)
corr_IV = sim_study(Q, cov, Type = "IV")


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

-22.054093599040627
P 7.999307595136476 0.016215530041058468
S 0.9086246815970481 2.6013819153943486e-17
-20.605716829592083
P 9.408532750549599 0.046304423160617114
S 0.7835080809547473 2.079278218593298e-20


In [356]:

cov = np.array([[1,0.0],[0.0,1]])
wo_corr = sim_study(Q,cov, Type = "OLS", Consts = False)
wo_corr_IV = sim_study(Q,cov, Type = "IV")


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

In [357]:
wo_corr_pd = pd.DataFrame(wo_corr)
wo_corr_pd_IV = pd.DataFrame(wo_corr_IV)
corr_pd = pd.DataFrame(corr)
corr_pd_IV = pd.DataFrame(corr_IV)


In [358]:
pd.DataFrame([wo_corr_pd.mean(axis=0),wo_corr_pd_IV.mean(axis=0), corr_pd.mean(axis=0), corr_pd_IV.mean(axis=0)])

,0,1,2,3
0,-5.001121,3.002007,1.996707,1.008017
1,-4.958159,2.955566,1.904352,0.865639
2,-4.714164,2.644749,1.377273,0.105250
3,-4.927514,2.914395,1.854408,0.765774


In [359]:
pd.DataFrame([wo_corr_pd.median(axis=0),wo_corr_pd_IV.median(axis=0), corr_pd.median(axis=0), corr_pd_IV.median(axis=0)])

,0,1,2,3
0,-5.000317,3.000446,1.992738,1.002896
1,-4.989477,2.981321,1.980974,0.977704
2,-4.715478,2.634855,1.370012,0.094551
3,-4.950295,2.903842,1.875029,0.827262


Results so far: 

With parameters 

0	1	2	3
0	-3.000049	1.004966	0.994198	1.995436
1	-2.992413	0.999941	0.978292	1.976096
2	-2.994373	0.978406	0.996163	2.002496
3	-3.002868	1.007717	0.992143	2.007656




In [ ]:
#corr_pd = corr_pd.loc[~(corr_pd==0).all(axis=1)]


fig, ax =plt.subplots(1,3)
sns.histplot(wo_corr_pd[0], ax=ax[0])
sns.histplot(wo_corr_pd[1], ax=ax[1]).set(title='OLS without Correlation')
sns.histplot(wo_corr_pd[2], ax=ax[2])


fig.show()

In [ ]:
#corr_pd = corr_pd.loc[~(corr_pd==0).all(axis=1)]


fig, ax =plt.subplots(1,3)
sns.histplot(corr_pd_IV[0], ax=ax[0])
sns.histplot(corr_pd_IV[1], ax=ax[1]).set(title='IV without Correlation')
sns.histplot(corr_pd_IV[2], ax=ax[2])


fig.show()

In [ ]:
#corr_pd = corr_pd.loc[~(corr_pd==0).all(axis=1)]

fig, ax =plt.subplots(1,3)
sns.histplot(corr_pd[0], ax=ax[0])
sns.histplot(corr_pd[1], ax=ax[1]).set(title='OLS with Correlation')
sns.histplot(corr_pd[2], ax=ax[2])

fig.show()

In [ ]:
wo_corr_pd.mean()

In [ ]:
#corr_pd = corr_pd.loc[~(corr_pd==0).all(axis=1)]

fig, ax =plt.subplots(1,3)
sns.histplot(corr_pd_IV[0], ax=ax[0])
sns.histplot(corr_pd_IV[1], ax=ax[1]).set(title='IV with Correlation')
sns.histplot(corr_pd_IV[2], ax=ax[2])

fig.show()

In [ ]:
corr_pd_IV.mean()

In [ ]:
corr_pd = pd.DataFrame(corr_IV)
corr_pd = corr_pd.loc[~(corr_pd==0).all(axis=1)]

fig, ax =plt.subplots(1,3)
sns.histplot(corr_pd[0], ax=ax[0])
sns.histplot(corr_pd[1], ax=ax[1])
sns.histplot(corr_pd[2], ax=ax[2])

fig.show()

In [ ]:
corr_pd.mean()

### Adding Instruments: 
BLP Instruments consist of: 
 - average attributes of own firm other products characteristics (if j = 1, then this instrument is invalid)
 - average attributes of other firm product characteristics.     


In [ ]:
# I need to first estimate a model consisting only of constants. 
# Then I need to regress these constants on instruments. 

### Results
Unexpected result since OLS is able to recover the parameters of the model exactly. Right now the model is estimated Must be because pr

In [ ]:
X.repeat(200, axis=0).shape

In [ ]:
X.repeat(200, axis=0).shape


In [ ]:
mean = [0,0]
cov  = np.array([[1,0.1],[0.1,1]])
test=np.random.multivariate_normal(mean,cov,size = (f,j) )
test.shape

In [ ]:
cov

In [ ]:
np.random.multivariate_normal([0,0],cov, size = (f*j)).shape

In [ ]:
x = np.array([1,1,1,0])
np.maximum(x, 0.5)

In [ ]:



def sim_study_IV(Q,cov):
    parameters = np.zeros((Q,3))#+4*2))
    for q in tqdm(range(Q)): 
        try:
            f=2 # Firms
            j=3 # Products
            k=2 # Traits (of each product) 

            data = DE.endog_data()
            X, prod=data.simulate_product_traits(f,j,k, which_type = "Differentiated")
            X = X.reshape(j*f, -1)
            H = data.hadamard(f*j,prod)

            # True Parameters 
            alpha = -1                # Price sensitivity
            beta= np.array([1,1])        # Preference for traits 
            theta = np.append(alpha,beta)  
            #cov = np.array([[1,0.5],[0.5,1]])

            gamma = -np.array([1,1,1])
            T = 20
            P0 = np.zeros((f*j)) +1               #Initital price vector

            # Simulate data 
            S = np.array(0) 
            while S.min() == 0.0:
                P, S = data.sim_data_T(X, f, j, T, theta, gamma, cov, prod, H, disable = True)
            S.min()
            
            # constructing instruments: 
            # Not implemented

            #Estimation: 
            S0=(1-np.sum(S,axis=0))

            index=np.log(S/S0).reshape(-1,1)
            

            check = np.array(np.arange(j*f))
            Z = []
            for firm, js in prod.items():
                not_f_prods=[j for j in check if j not in js ]

                Z_i = X[not_f_prods,:].mean(axis=0)

                Z.append(Z_i)

            Z = np.array(Z) 

            ZZ = Z.repeat(T, axis=0)
            Piv1 = P.reshape(-1)


            results1step=OLS(Piv1,ZZ).fit()
            results1step.params

            EP = (ZZ @ results1step.params).reshape(-1,1)

            # Step 2 IV: 
            ZX = np.hstack((EP, X.repeat(T, axis=0)))
            #XX = XX[:,0:2]
            resultsIV=OLS(index,ZX).fit()
        
            parameters[q,:] = resultsIV.params
            
        except np.linalg.LinAlgError: 
            continue
    return parameters




    